# 이미지 리사이즈

In [ ]:
def resize_image(input_image_path, output_image_path, size):
    original_image = Image.open(input_image_path)
    width, height = original_image.size
    aspect_ratio = float(width) / float(height)
    new_height = size[0]
    new_width = int(aspect_ratio * new_height)
    resized_image = original_image.resize((new_width, new_height))
    resized_image.save(output_image_path)

# 하위 폴더의 이미지들을 읽어들여 리사이즈하는 함수
def resize_images_in_folder(folder_path, output_folder_path, size):
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for filename in os.listdir(folder_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            input_image_path = os.path.join(folder_path, filename)
            output_image_path = os.path.join(output_folder_path, filename)
            resize_image(input_image_path, output_image_path, size)

# xml 리사이즈 및 리네임

In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET

def replace_name_in_xml(xml_file_path):
    # XML 파일 읽기
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    # 모든 'name' 태그 찾기
    name_tags = root.findall(".//name")

    # 'name' 태그 값 변경
    for name_tag in name_tags:
        filename = os.path.basename(xml_file_path)
        name = filename[:5]  # 파일명의 맨 앞 5자리 숫자로 변경
        name_tag.text = name
    
    # 변경된 XML 파일 저장
    tree.write(xml_file_path, encoding="utf-8")

def copy_and_modify_xml_files(input_dir, output_dir):
    # 출력 디렉토리 생성
    os.makedirs(output_dir, exist_ok=True)

    # 입력 디렉토리의 XML 파일들 복사 및 수정
    for filename in os.listdir(input_dir):
        if filename.endswith(".xml"):
            xml_file_path = os.path.join(input_dir, filename)
            replace_name_in_xml(xml_file_path)
            output_path = os.path.join(output_dir, filename)
            shutil.copy(xml_file_path, output_path)

In [ ]:
import os
import xml.etree.ElementTree as ET
import shutil

def modify_bounding_box(xml_folder, save_dir):
    # 폴더 내의 XML 파일들 순회
    for xml_file in os.listdir(xml_folder):
        if xml_file.endswith('.xml'):
            # XML 파일 경로
            xml_path = os.path.join(xml_folder, xml_file)
            
            # XML 파일 로드
            tree = ET.parse(xml_path)
            root = tree.getroot()
            
            # <size> 요소의 <width>와 <height> 수정
            size_elem = root.findall('.//size')[0]
            
            origin_width = int(size_elem.find('width').text)
            origin_height = int(size_elem.find('height').text)
            
            size_elem.find('width').text = '1000'
            size_elem.find('height').text = '1000'

            # <bndbox> 좌표 변경
            for bbox in root.iter('bndbox'):
                xmin = int(bbox.find('xmin').text)
                ymin = int(bbox.find('ymin').text)
                xmax = int(bbox.find('xmax').text)
                ymax = int(bbox.find('ymax').text)

                # 좌표 조정
                xmin = int(xmin * (1000 / origin_width))
                ymin = int(ymin * (1000 / origin_height))
                xmax = int(xmax * (1000 / origin_width))
                ymax = int(ymax * (1000 / origin_height))

                # 조정된 좌표 반영
                bbox.find('xmin').text = str(xmin)
                bbox.find('ymin').text = str(ymin)
                bbox.find('xmax').text = str(xmax)
                bbox.find('ymax').text = str(ymax)

            # 변경된 XML 파일 저장
            save_path = os.path.join(save_dir, xml_file)4
            tree.write(save_path, encoding='utf-8')

# 이미지 합치기

## 1x2

In [ ]:
from PIL import Image

def merge_images(image_a_path, image_b_path, output_path):
    # 이미지 A 로드
    image_a = Image.open(image_a_path)
    
    # 이미지 B 로드
    image_b = Image.open(image_b_path)
    
    # 이미지 A와 B의 크기가 동일한지 확인
    if image_a.size != image_b.size:
        raise ValueError("이미지 A와 B의 크기가 다릅니다.")
    
    # 합쳐질 이미지 생성
    merged_image = Image.new('RGB', (image_a.width + image_b.width, image_a.height))
    
    # 이미지 A를 왼쪽에 복사
    merged_image.paste(image_a, (0, 0))
    
    # 이미지 B를 오른쪽에 복사
    merged_image.paste(image_b, (image_a.width, 0))
    
    # 합쳐진 이미지 저장
    merged_image.save(output_path)

## 2x2

In [1]:
from PIL import Image

def merge_images_2x2(image_a_path, image_b_path, image_c_path, image_d_path, output_path):
    # 이미지 A, B, C, D 불러오기
    image_a = Image.open(image_a_path)
    image_b = Image.open(image_b_path)
    image_c = Image.open(image_c_path)
    image_d = Image.open(image_d_path)

    # 이미지 크기가 동일한지 확인
    if image_a.size != image_b.size or image_a.size != image_c.size or image_a.size != image_d.size:
        raise ValueError("이미지 A, B, C, D의 크기가 모두 동일하지 않습니다.")

    # 2x2 그리드로 합칠 이미지 생성
    merged_image = Image.new('RGB', (image_a.width * 2, image_a.height * 2))

    # 이미지 A를 좌상단에 복사
    merged_image.paste(image_a, (0, 0))

    # 이미지 B를 우상단에 복사
    merged_image.paste(image_b, (image_a.width, 0))

    # 이미지 C를 좌하단에 복사
    merged_image.paste(image_c, (0, image_a.height))

    # 이미지 D를 우하단에 복사
    merged_image.paste(image_d, (image_a.width, image_a.height))

    # 바운딩 박스 좌표를 합쳐진 이미지 크기에 맞게 조정
    width_diff = image_a.width
    height_diff = image_a.height

    merged_image.save(output_path)

# xml 합치기(2x2)

## 1x2

In [ ]:
import os
import xml.etree.ElementTree as ET

def adjust_bbox_coordinates(bbox, width_diff):
    # 바운딩 박스 좌표를 조정하는 함수
    xmin = int(bbox.find('xmin').text)
    xmax = int(bbox.find('xmax').text)

    bbox.find('xmin').text = str(xmin + width_diff)
    bbox.find('xmax').text = str(xmax + width_diff)

def merge_xml_files(xml_a_path, xml_b_path, output_path, new_filename):
    # XML 파일 A 로드
    tree_a = ET.parse(xml_a_path)
    root_a = tree_a.getroot()

    # XML 파일 B 로드
    tree_b = ET.parse(xml_b_path)
    root_b = tree_b.getroot()

    # 이미지 B 파일명 추출
    filename_b = root_b.find('filename').text

    # 이미지 A의 가로 크기와 이미지 B의 가로 크기 계산
    width_a = int(root_a.find('size/width').text)
    width_b = int(root_b.find('size/width').text)
    width_diff = width_a

    # 이미지 B의 바운딩 박스들에 대해 좌표 조정
    for obj in root_b.findall('object'):
        adjust_bbox_coordinates(obj.find('bndbox'), width_diff)
        # 이미지 A에 있는 모든 bbox를 가져와서 새로운 Element 객체로 추가
        root_a.append(obj)

    # 이미지 B를 이미지 A 오른쪽에 합치기
    root_a.find('filename').text = new_filename + '.jpg'
    root_a.find('size/width').text = str(width_a + width_b)

    # 새로운 XML 파일 저장
    tree_a.write(output_path, encoding='utf-8')

## 2x2

In [ ]:
import os
import xml.etree.ElementTree as ET

def adjust_bbox_coordinates(bbox, width_diff, height_diff):
    # 바운딩 박스 좌표를 조정하는 함수
    xmin = int(bbox.find('xmin').text)
    xmax = int(bbox.find('xmax').text)
    ymin = int(bbox.find('ymin').text)
    ymax = int(bbox.find('ymax').text)

    bbox.find('xmin').text = str(xmin + width_diff)
    bbox.find('xmax').text = str(xmax + width_diff)
    bbox.find('ymin').text = str(ymin + height_diff)
    bbox.find('ymax').text = str(ymax + height_diff)

def merge_xml_files(xml_a_path, xml_b_path, xml_c_path, xml_d_path, output_path, new_filename):
    # XML 파일 A 로드
    tree_a = ET.parse(xml_a_path)
    root_a = tree_a.getroot()

    # 이미지 A의 가로 크기와 이미지 B, C, D의 가로 크기 계산
    width_a = int(root_a.find('size/width').text)
    tree_b = ET.parse(xml_b_path)
    root_b = tree_b.getroot()
    width_b = int(root_b.find('size/width').text)
    tree_c = ET.parse(xml_c_path)
    root_c = tree_c.getroot()
    width_c = int(root_c.find('size/width').text)
    tree_d = ET.parse(xml_d_path)
    root_d = tree_d.getroot()
    width_d = int(root_d.find('size/width').text)

    # 이미지 A의 세로 크기와 이미지 B, C, D의 세로 크기 계산
    height_a = int(root_a.find('size/height').text)
    height_b = int(root_b.find('size/height').text)
    height_c = int(root_c.find('size/height').text)
    height_d = int(root_d.find('size/height').text)

    # 이미지 B의 바운딩 박스들에 대해 좌표 조정
    for obj in root_b.findall('object'):
        adjust_bbox_coordinates(obj.find('bndbox'), width_a, 0)
        # 이미지 A에 있는 모든 bbox를 가져와서 새로운 Element 객체로 추가
        root_a.append(obj)

    # 이미지 C의 바운딩 박스들에 대해 좌표 조정
    for obj in root_c.findall('object'):
        adjust_bbox_coordinates(obj.find('bndbox'), 0, height_a)
        # 이미지 A에 있는 모든 bbox를 가져와서 새로운 Element 객체로 추가
        root_a.append(obj)

    # 이미지 D의 바운딩 박스들에 대해 좌표 조정
    for obj in root_d.findall('object'):
        adjust_bbox_coordinates(obj.find('bndbox'), width_a, height_a)
        # 이미지 A에 있는 모든 bbox를 가져와서 새로운 Element 객체로 추가
        root_a.append(obj)
        
    root_a.find('filename').text = new_filename + '.jpg'
    
    # 새로운 XML 파일 저장
    tree_a.write(output_path, encoding='utf-8')